# How to use the LabelGenerator classes with timebased labelling

## 1. Setup

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import warnings
warnings.filterwarnings("ignore")

In [9]:
# Make sure you are in the drone_steering directory

#os.chdir('..')
os.getcwd()

'C:\\Users\\p.schambach\\Desktop\\DSR\\drone_steering'

In [24]:
data_dir = "data/gesture/"
file_name_label = "labels_flip_p_01.csv"

In [11]:
df = pd.read_csv(data_dir + file_name)

In [12]:
df

,real_start,real_end,diff,label
0,18.70,20.45,-0.25,5
1,20.40,22.40,0.00,5
2,22.35,24.50,0.15,5
3,26.65,28.55,-0.10,5
4,30.70,32.60,-0.10,5
5,34.65,36.35,-0.30,5
6,38.65,40.55,-0.10,5
7,42.60,44.50,-0.10,5
8,46.50,48.40,-0.10,5
9,50.70,52.50,-0.20,5


In [27]:
file_name_feature = "features_flip_p_01_120.csv"
feature_data = pd.read_csv(data_dir + file_name_feature)
feature_data.head()

,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y
0,364.259742,233.104313,341.540132,283.689744,346.400313,167.802496,372.988320,284.164713,270.588044,221.382945,294.221199,285.467308,277.812103,166.122200,269.901853,275.441969
1,363.465416,232.195799,343.133862,284.560465,347.432609,165.740016,373.414323,284.624033,269.537224,224.415170,295.345827,286.248028,277.530304,166.118435,269.124248,277.101146
2,358.235418,229.564618,336.989160,287.526567,346.002612,167.452371,372.976597,286.866511,272.654944,226.725110,293.317980,287.062334,280.954640,167.564055,265.391056,276.457711
3,362.473412,231.022458,343.309504,283.651798,349.276440,166.365935,371.782201,284.483968,269.523653,224.547709,295.074371,284.941882,277.163612,166.128539,269.328065,276.668479
4,364.180072,231.006023,344.422331,283.465604,347.642222,165.250143,372.912682,285.427752,270.456436,220.697018,296.207271,283.810974,277.393757,165.318363,268.442911,278.002018


## 2. Usage of the LabelGenerator class

In [28]:
from app_local.module import LabelGenerator

In [29]:
lgen = LabelGenerator(
    # This is supposed to be your wireframe data from posenet
    data = feature_data,

    # manually labeled "raw" gesture data with real beginning and real end of movement
    raw_labels = df[["real_start","real_end","label"]],
    
    # associated frame rate of the LabelGenerator. This instance only works with the framerate specified on instantiation
    ms_per_frame = 130    
)

lgen.fit_range(
    # the length of the tolerance range will be 400 ms
    tolerance_range = 400,
    
    # maximum acceptable difference/error in movement length compared to the theoretical movement length (2000 ms)
    # if movement length is smaller than 2000 - max_error or greater than 2000 + max_error, there will be 0-label
    max_error = 400
)

# creates the labeled data set. it can be called with the get_labeled_data method
lgen.set_labels()

# provides 3D labeled data and labels for training. The instance can call X, y, feature_names and final_time
# X --> Array with dimensions [sample size] x [timesteps per sample] x [number of features]
# y --> vector of labels with length [sample size]
# feature_names --> list of the names of the assiciated columns in X
# sequence_end_time --> vector with the number of milliseconds associated with the first dimension of X ([sample size])
lgen.extract_training_data()

In [30]:
# label_df stores information about the labelled time invervals
# if the length deviates too much from 2 seconds, the interval is ignored and not used for training
lgen.label_df

,label,real_start,real_end,from,to,ignore
0,5,18700,20450,20125,20525,False
1,5,20400,22400,22200,22600,False
2,5,22350,24500,24375,24775,False
3,5,26650,28550,28300,28700,False
4,5,30700,32600,32350,32750,False
5,5,34650,36350,36000,36400,False
6,5,38650,40550,40300,40700,False
7,5,42600,44500,44250,44650,False
8,5,46500,48400,48150,48550,False
9,5,50700,52500,52200,52600,False


In [31]:
# label_info contains information about the deviation from the target 2 second length
# upper slack - lower slack is always equal to the tolerance range (here: 400 ms)
lgen.label_info

,diff,l_slack,u_slack,tol_range_indicator
0,-250,-325,75,True
1,0,-200,200,True
2,150,-125,275,True
3,-100,-250,150,True
4,-100,-250,150,True
5,-300,-350,50,True
6,-100,-250,150,True
7,-100,-250,150,True
8,-100,-250,150,True
9,-200,-300,100,True


In [32]:
# labeled data contains all datapoints with the correct labels (or no label at all --> label "0")
lgen.labeled_data.head()

,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y,label,time
0,364.259742,233.104313,341.540132,283.689744,346.400313,167.802496,372.988320,284.164713,270.588044,221.382945,294.221199,285.467308,277.812103,166.122200,269.901853,275.441969,0,130
1,363.465416,232.195799,343.133862,284.560465,347.432609,165.740016,373.414323,284.624033,269.537224,224.415170,295.345827,286.248028,277.530304,166.118435,269.124248,277.101146,0,260
2,358.235418,229.564618,336.989160,287.526567,346.002612,167.452371,372.976597,286.866511,272.654944,226.725110,293.317980,287.062334,280.954640,167.564055,265.391056,276.457711,0,390
3,362.473412,231.022458,343.309504,283.651798,349.276440,166.365935,371.782201,284.483968,269.523653,224.547709,295.074371,284.941882,277.163612,166.128539,269.328065,276.668479,0,520
4,364.180072,231.006023,344.422331,283.465604,347.642222,165.250143,372.912682,285.427752,270.456436,220.697018,296.207271,283.810974,277.393757,165.318363,268.442911,278.002018,0,650


In [33]:
# the training data is stored in X (array of 3 dimensions; [sample size] x [timesteps per sample] x [number of features])
# the label-vector is y (1 dim; [sample size])
print(len(lgen.y))
print(lgen.X.shape)
print("")

i = 157
print(lgen.y[i], '\n', lgen.X[i,:,:])

533
(533, 16, 16)

5.0 
 [[331.1244619   94.27365232 297.7164934  257.7552323  295.4847851
  134.801131   338.3379625   12.8189239  226.7236441  214.2339882
  253.5553804  271.2860862  228.8366327  152.5611822  221.2712785
  260.5884571 ]
 [336.7924066  102.5173049  298.5817971  262.2727309  302.8673442
  135.6657852  374.8685899   56.74949775 224.3160687  214.3038199
  249.6319378  267.8861076  232.1321474  150.7230333  220.8353268
  262.4058343 ]
 [347.7648197  115.8507682  299.9228464  258.6924488  300.3766921
  138.1202606  410.2073965   75.85174832 219.6341976  208.4268503
  250.5995947  271.3220256  227.3626353  147.9675961  221.7177717
  260.9890032 ]
 [346.1152046  133.928921   298.9516854  259.7046262  299.6932797
  138.2569607  408.5363284  117.3916414  221.3202665  204.4288418
  249.158789   271.4964246  224.6516172  147.2897882  216.0455759
  260.7037989 ]
 [358.3216465  165.129359   295.9610827  262.6585467  300.3932963
  142.0628715  420.4037727  172.8472333  217.5628443 

In [34]:
# sequence_end_time stored the last sample timestamp in ms, 
#   when a sequence of 2000ms/130ms (sequence length / framelength) is complete

# first sequence ends with the sample with timestamp 2080 ms
#  then it shifts by 130 ms (the framelength)
lgen.sequence_end_time[0:10]

array([2080, 2210, 2340, 2470, 2600, 2730, 2860, 2990, 3120, 3250])

In [35]:
lgen.labeled_data.tail(2)

,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y,label,time
546,267.727891,263.289408,240.486626,317.246946,248.086896,198.001601,260.013920,320.000638,177.124243,262.897095,205.907800,314.551857,187.102613,199.389253,170.411104,316.178173,0,71110
547,267.876649,270.066788,239.650008,317.936445,246.387812,198.586740,260.892048,316.925162,173.321533,262.630270,205.941307,316.733591,188.036930,199.995727,163.221029,313.946029,0,71240


In [36]:
# the feature names are also stored:
lgen.feature_names

Index(['leftElbow_x', 'leftElbow_y', 'leftHip_x', 'leftHip_y',
       'leftShoulder_x', 'leftShoulder_y', 'leftWrist_x', 'leftWrist_y',
       'rightElbow_x', 'rightElbow_y', 'rightHip_x', 'rightHip_y',
       'rightShoulder_x', 'rightShoulder_y', 'rightWrist_x', 'rightWrist_y'],
      dtype='object')